## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import itertools
import math
from pprint import pprint
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder

# To execute a cell line by line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

# Decision Trees

Consider the problem of predicting if a person has a college degree based on age and salary. The table
and graph below contain training data for 10 individuals. Now build a decision tree for classifying whether a person has a college degree by greedily choosing threshold splits that maximize information
gain. What is the depth of your tree and the information gain at each split?

In [2]:
# Loading data

Age = np.array([24, 53, 23, 25, 32, 52, 22, 43, 52, 48])
Salary = np.array([40000, 52000, 25000, 77000, 48000, 110000, 38000, 44000, 27000, 65000])
College_Degree = np.array([1, 0, 0, 1, 1, 1, 1, 0, 0, 1])
df = pd.DataFrame(list(zip(Age, Salary, College_Degree)), columns = ['Age', 'Salary ($)', 'College Degree'])
df

,Age,Salary ($),College Degree
0,24,40000,1
1,53,52000,0
2,23,25000,0
3,25,77000,1
4,32,48000,1
5,52,110000,1
6,22,38000,1
7,43,44000,0
8,52,27000,0
9,48,65000,1


In [65]:
cols = ['Age','Salary ($)']
X = np.array(list(zip(Age,Salary)))
y = College_Degree

### 1) Decision tree implementation from scratch

In [66]:
# Entropy calculation

def entropy(cls, tot):
  return -(cls*1.0/tot)*math.log(cls*1.0/tot, 2)

def entropyCalc(c1, c2):
  if c1== 0 or c2 == 0:  # when there is only one class in the group, entropy is 0
    return 0
  return entropy(c1, c1+c2) + entropy(c2, c1+c2)

def entropyDiv(div): 
  s = 0
  n = len(div)
  classes = set(div)
  for c in classes:   # for each class, get entropy
    n_c = sum(div==c)
    e = n_c*1.0/n * entropyCalc(sum(div==c), sum(div!=c)) # weighted avg
    s += e
  return s, n

def get_entropy(y_predict, y_real):
  if len(y_predict) != len(y_real):
    print('They have to be the same length')
    return None
  n = len(y_real)
  s_true, n_true = entropyDiv(y_real[y_predict]) # left hand side entropy
  s_false, n_false = entropyDiv(y_real[~y_predict]) # right hand side entropy
  entropy = n_true*1.0/n * s_true + n_false*1.0/n * s_false # overall entropy, again weighted average
  return entropy

In [81]:
class DecisionTreeAlgo():

  def __init__(self, max_depth):
    self.depth = 0
    self.max_depth = max_depth
    
  def fit(self, x, y, node={}, depth=0):
    if node is None: 
      return None
    elif len(y) == 0:
      return None
    elif self.all_same(y):
      return {'val':y[0]}
    elif depth >= self.max_depth:
        return None
    else: 
        col, cutoff, entropy = self.allBestSplit(x, y)    # find one split given an information gain
        print("Information gain : ", round(info_gain,2))
        y_left = y[x[:, col] < cutoff]
        y_right = y[x[:, col] >= cutoff]
        node = {'col': cols[col], 'index_col':col,'cutoff':cutoff,'val': np.round(np.mean(y))}
        node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)
        node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)
        self.depth += 1 
        self.trees = node
        return node
    
  def allBestSplit(self, x, y):
    col = None
    min_entropy = 1
    cutoff = None
    for i, c in enumerate(x.T):
        entropy, cur_cutoff = self.bestSplit(c, y)
        if entropy == 0:    # find the first perfect cutoff. Stop Iterating
            return i, cur_cutoff, entropy
        elif entropy <= min_entropy:
            min_entropy = entropy
            col = i
            cutoff = cur_cutoff
    return col, cutoff, min_entropy

  def bestSplit(self, col, y):
    min_entropy = 10
    n = len(y)
    for value in set(col):
        y_predict = col < value
        my_entropy = get_entropy(y_predict, y)
        if my_entropy <= min_entropy:
            min_entropy = my_entropy
            cutoff = value
    return min_entropy, cutoff
  
  def all_same(self, items):
    return all(x == items[0] for x in items)
                                           
  def predict(self, x):
    tree = self.trees
    results = np.array([0]*len(x))
    for i, c in enumerate(x):
        results[i] = self.prediction(c)
    return results
  
  def prediction(self, row):
    cur_layer = self.trees
    while cur_layer.get('cutoff'):
        if row[cur_layer['index_col']] < cur_layer['cutoff']:
            cur_layer = cur_layer['left']
        else:
            cur_layer = cur_layer['right']
    else:
        return cur_layer.get('val')

In [82]:
clf = DecisionTreeAlgo(max_depth=7)
tree = clf.fit(X, y)
pprint(tree)

Information gain :  0.35
Information gain :  0.5
Information gain :  1.0
{'col': 'Salary ($)',
 'cutoff': 38000,
 'index_col': 1,
 'left': {'val': 0},
 'right': {'col': 'Age',
           'cutoff': 43,
           'index_col': 0,
           'left': {'val': 1},
           'right': {'col': 'Salary ($)',
                     'cutoff': 65000,
                     'index_col': 1,
                     'left': {'val': 0},
                     'right': {'val': 1},
                     'val': 0.0},
           'val': 1.0},
 'val': 1.0}


Here, we observe that the depth of the tree is 3

### 2) Multivariate decision tree

A multivariate decision tree is a generalization of univariate decision trees, where more than one
attribute can be used in the decision rule for each split. That is, splits need not be orthogonal to a
feature’s axis.
For the same data, learn a multivariate decision tree where each decision rule is a linear classifier that
makes decisions based on the sign of αxage + βxincome − 1. Draw your tree, including the α, β and the
information gain for each split.


CART algorithm for decisions tree can be made into a Multivariate.To create a Multivariant tree we need to compute the best split at each node, but instead of throwing away all splits that weren't the best we take a portion of those (maybe all), then evaluate all of the data's attributes by each of the potential splits at that node weighted by the order. So the first split (which lead to the maximum gain) is weighted at 1. Then the next highest gain split is weighted by some fraction < 1.0, and so on. Where the weights decrease as the gain of that split decreases. That number is then compared to same calculation of the nodes within the left node if it's above that number go left. Otherwise go right. This can be a multi-variant split for decision trees. Adding multivariate splits expands the search space enormously.

### 3) Multivariate decision trees vs Univariate decision trees

Advantages

1) Univariate means involving one variate or variable quantity. On the other hand, multivariate means using multiple or more than one variate or variable quantity at the same time. Therefore, multivariate decision trees can use use all attributes at one node when branching

2) If the data set is small or if one class has a signiﬁcantly larger number of instancescompared to other classes, then a univariate technique does not overﬁt and can be sufficient






Disadvantages

1) Hyperplanes with an arbitrary orientation are used in multivariate trees. It means that there can be 2^d {N choose d} possible Hyperplanes (where d is the number of dimensions and N is the number of possible thresholds for the split points) which makes exhaustive search inefficient and impractical. Consequently, a more practical way to follow is using linear multivariate node that takes weights for each attribute and sums them up . Moreover, linear multivariate decision trees choose the most important attributes amongst all so that the process would become more efficient and practical.

2) If the variables are highly correlated, then a univariate method is not sufficient and we may resort to multivariate methods.